In [1]:
import pandas as pd
import covasim as cv

from util import comb

# Covasim

## Running a simulation

In [2]:
pars = dict(
  pop_type='hybrid',
  pop_size=10000
)

sim = cv.Sim(pars)

Covasim 2.0.2 (2020-02-01) — © 2021 by IDM


In [3]:
intervention_pars = [
 'beta',         # beta per symptomatic contact
 'contacts',     # the number of contacts per layer
 'dynam_layer',  # which layers are dynamic
 'beta_layer',   # transmissibility per layer
 'n_imports',    # average daily number of imported cases (actual number is drawn from Poisson distribution)
 'asymp_factor', # multiply beta by this factor for asymptomatic cases; no statistically significant difference in transmissibility
 'iso_factor',   # multiply beta by this factor for diganosed cases to represent isolation
 'quar_factor',  # quarantine multiplier on transmissibility and susceptibility
 'quar_period',  # number of days to quarantine for
 'interventions' # the interventions present in this simulation
]

for key in intervention_pars:
  print(f'{key:13} --> {sim[key]}')

beta          --> 0.016
contacts      --> {'h': 2.0, 's': 20, 'w': 16, 'c': 20}
dynam_layer   --> {'h': 0, 's': 0, 'w': 0, 'c': 0}
beta_layer    --> {'h': 3.0, 's': 0.6, 'w': 0.6, 'c': 0.3}
n_imports     --> 0
asymp_factor  --> 1.0
iso_factor    --> {'h': 0.3, 's': 0.1, 'w': 0.1, 'c': 0.1}
quar_factor   --> {'h': 0.6, 's': 0.2, 'w': 0.2, 'c': 0.2}
quar_period   --> 14
interventions --> []


The execution of a simulation produces the following results:

In [4]:
sim.run()

Initializing sim with 10000 people for 60 days
  Running 2020-03-01 ( 0/60) (0.41 s)  ———————————————————— 2%
  Running 2020-03-11 (10/60) (0.46 s)  •••————————————————— 18%
  Running 2020-03-21 (20/60) (0.52 s)  ••••••—————————————— 34%
  Running 2020-03-31 (30/60) (0.58 s)  ••••••••••—————————— 51%
  Running 2020-04-10 (40/60) (0.65 s)  •••••••••••••——————— 67%
  Running 2020-04-20 (50/60) (0.72 s)  ••••••••••••••••———— 84%
  Running 2020-04-30 (60/60) (0.80 s)  •••••••••••••••••••• 100%

Simulation summary:
    6342 cumulative infections
    5377 cumulative infectious
       0 cumulative tests
       0 cumulative diagnoses
    3066 cumulative recoveries
    3494 cumulative symptomatic cases
     205 cumulative severe cases
      49 cumulative critical cases
       9 cumulative deaths
       0 cumulative quarantined people



## Running multiple simulations

In [5]:
multi_pars = [ (k, v) for k, v in dict(
  rand_seed = range(3),
  beta = [0.01, 0.02, 0.03]
).items()]

multi_pars

[('rand_seed', range(0, 3)), ('beta', [0.01, 0.02, 0.03])]

In [6]:
pars['verbose'] = 0
pars_list = comb.cartesian_product(multi_pars, pars)
pars_list

[{'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 0,
  'beta': 0.01},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 0,
  'beta': 0.02},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 0,
  'beta': 0.03},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 1,
  'beta': 0.01},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 1,
  'beta': 0.02},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 1,
  'beta': 0.03},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 2,
  'beta': 0.01},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 2,
  'beta': 0.02},
 {'pop_type': 'hybrid',
  'pop_size': 10000,
  'verbose': 0,
  'rand_seed': 2,
  'beta': 0.03}]

In [7]:
sims = [cv.Sim(pars, label=f'Sim {i}') for i, pars in enumerate(pars_list)]
msim = cv.MultiSim(sims)
msim.brief()

MultiSim(n_sims: 9; base: Sim("Sim 0"; 2020-03-01 to 2020-04-30; pop: 10000 hybrid; epi: not run))


In [8]:
%%time
msim.run()

CPU times: user 60 ms, sys: 32 ms, total: 92.1 ms
Wall time: 3.79 s


In [9]:
keys = [key for key in msim.sims[0].summary.keys() if 'cum' in key]
values = []

for sim in msim.sims:
  values.append(sim.summary[keys])
values

[[1653.0, 1358.0, 0.0, 0.0, 917.0, 919.0, 53.0, 12.0, 2.0, 0.0],
 [8280.0, 7827.0, 0.0, 0.0, 5824.0, 5174.0, 354.0, 79.0, 24.0, 0.0],
 [9367.0, 9356.0, 0.0, 0.0, 9123.0, 6332.0, 524.0, 141.0, 55.0, 0.0],
 [1164.0, 945.0, 0.0, 0.0, 569.0, 621.0, 31.0, 9.0, 1.0, 0.0],
 [8266.0, 7686.0, 0.0, 0.0, 5449.0, 5103.0, 363.0, 89.0, 17.0, 0.0],
 [9376.0, 9348.0, 0.0, 0.0, 8990.0, 6306.0, 520.0, 146.0, 60.0, 0.0],
 [2908.0, 2414.0, 0.0, 0.0, 1518.0, 1580.0, 83.0, 20.0, 4.0, 0.0],
 [8810.0, 8664.0, 0.0, 0.0, 7650.0, 5846.0, 433.0, 108.0, 35.0, 0.0],
 [9493.0, 9487.0, 0.0, 0.0, 9282.0, 6363.0, 505.0, 149.0, 46.0, 0.0]]

## Generate the Dataset

Information about hospital capacities has been retrieved from [Wikipedia](https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds).

In [ ]:
fixed_params = {}
fixed_params['pop_type'] = 'hybrid'                                  # hybrid population type (four default contact layers included)
fixed_params['location'] = 'italy'                                   # households and age distribution data from italy
fixed_params['n_days'] = 180                                         # 6-months simulation
fixed_params['pop_size'] = 50e3                                      # 50k agents in total
fixed_params['pop_infected'] = fixed_params['pop_size'] / 1e3        # 1 initial infected per 1k agents
fixed_params['n_beds_hosp'] = fixed_params['pop_size'] * 3.18 / 1e3  # 3.18 hospital beds per 1k citizens
fixed_params['n_beds_icu'] = fixed_params['pop_size'] * 12.5 / 100e3 # 12.5 hospital beds per 100k citizens
fixed_params['verbose'] = 0                                          # suppress prints

multi_params = [ (k, v) for k, v in dict(
  rand_seed = range(3),                                              # 3 simulations per input values
  n_imports = [5, 10],                                               # number of imported index cases
  quar_period = [7, 14],                                             # length of the quarantine period in days
).items() ]

In [ ]:
params_list = comb.cartesian_product(multi_params, fixed_params)
simulations = [cv.Sim(params, label=f'Sim {i}') for i, params in enumerate(params_list)]
multi_sim = cv.MultiSim(simulations)
multi_sim.run()

input_keys = [key for key, _ in multi_params]
output_keys = [key for key in multi_sim.sims[0].summary.keys() if 'cum' in key]
values = []
for i, simulation in enumerate(multi_sim.sims):
  input_values = [params_list[i][key] for key in input_keys]
  output_values = simulation.summary[output_keys]
  values.append(input_values + output_values)
df = pd.DataFrame(values, columns=input_keys + output_keys)

In [ ]:
df